In [ ]:
%%time
print("starting")
# from checkra import happyscribe_scrape as happy

main = "https://www.happyscribe.com"                
url="/public/lex-fridman-podcast-artificial-intelligence-ai"
html_location = "Archive/happyscribe_Lex_Fridman_html.txt"

# happy.update_transcripts(html_location, main)

In [1]:
%%time
from checkra.text_clean import text_fix, strip_accents

import spacy
import os
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import re
from checkra.nlp_pipeline import nlp
from checkra.topics.confidence_model import load_confidences, full_condense, generate_model

doc = nlp(text_fix(open("3Lex/#103|Ben_Goertzel|Artificial_General_Intelligence.txt").read()))
sents = [sent.text for sent in doc.sents]


# print(doc.user_data.keys())
dictionary, model = generate_model(doc.text, 2700)
condensed = load_confidences(doc.text, dictionary, model, sents) #generate initial topics + confidences, then smooth by filling in empty values and averaging
print(len(condensed))
fill = full_condense(condensed, len(sents), len(doc))
for ind, i in enumerate(fill):
    print(ind, i)

/ilab/users/hs884/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /ilab/users/hs884/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


634
0 [1.0, 176, [0, 176]]
1 [6.0, 362, [176, 538]]
2 [8.0, 191, [538, 729]]
3 [0.0, 282, [729, 1011]]
4 [5.0, 282, [1011, 1293]]
5 [8.0, 128, [1293, 1421]]
6 [3.0, 207, [1421, 1628]]
7 [6.0, 178, [1628, 1806]]
8 [7.0, 192, [1806, 1998]]
9 [0.0, 177, [1998, 2175]]
10 [9.0, 190, [2175, 2365]]
11 [4.0, 215, [2365, 2580]]
12 [8.0, 78, [2580, 2658]]
CPU times: user 28.8 s, sys: 5.31 s, total: 34.1 s
Wall time: 41.2 s


In [2]:
import numpy as np
from scipy import stats
import gensim
from gensim import corpora, models
from checkra.topics.preprocess import parallel_process, preprocess
import math




In [3]:

def load_confidences(file, dictionary, model, sents):#generates array with topic & confidence for each sentence
    sent_topics = np.zeros(len(sents))
    for i in range(len(sent_topics)): #for each sentence assign a topic
        bow_vector=dictionary.doc2bow(preprocess(sents[i]))
        try:
            sent_pred = sorted(model[bow_vector], key=lambda tup: -1*tup[1])[0]
            if sent_pred[1]>.54995 and sents[i].count(" ")>3: #adds only very confident sentences to list and sents with atleast 3 words
                sent_topics[i] = sent_pred[0]
            else:
                sent_topics[i] = -1
        except:
            print("touched")
            sent_topics[i] = -1
            
    i = 0
    while i <len(sent_topics): #fill in values for sentences with no topic using neighborhood
        if sent_topics[i]==-1: #if sentence doesn't have a topic
            interval = 3
            nhood = np.array([a for a in sent_topics[max(0, i-interval):min(len(sents), i+interval)] if a!=-1])
            
            while len(nhood)==0: #incase entire interval is empty, increase interval range
                interval+=1
                nhood =  np.array([a for a in sent_topics[max(0, i-interval):min(len(sents), i+interval)] if a!=-1])#neighborhood of 6 points around current point
            sent_topics[i] = statistics.mode(nhood)
            
        i+=1
    stream_data = []
    i = 0
    while i<len(sent_topics):
        stream_data.append([sent_topics[i], 1, [i,i+1]]) #(inclusive, exclusive)
        i+=1
        
    return stream_data


def condense_stream(sent_data):
    i = 0
    while i<len(sent_data)-1: #collect length of same topics that occur together
        if sent_data[i][0] == sent_data[i+1][0]: #if same as previous topic
            sent_data[i][2][1] = sent_data[i+1][2][1]
            sent_data[i][1] = sent_data[i][2][1] - sent_data[i][2][0]
            sent_data.pop(i+1)
        else:
            i+=1
    return sent_data

def trim_fill_gaps(sent_data, trim, sent_count, condense=lambda a: a):
    
    trimmed = [section for section in sent_data if section[1]>trim] 
    #keep sections of atleast a certain number of repeats
    

    trimmed[0][-1][0] = 0 # stretch first topic to beginning of podcast
    trimmed[0][1] = trimmed[0][-1][1] - trimmed[0][-1][0]
    i = 0
    while i <(len(trimmed)-1): #fill gaps between filtered numbers by readjusting counts and ranges
        if trimmed[i][2][1]!=trimmed[i+1][2][0]:
            diff = (trimmed[i+1][2][0]-trimmed[i][2][1])/2
    

            trimmed[i+1][2][0] -= int(diff)
            trimmed[i][2][1] += math.ceil(diff)

            trimmed[i+1][1] = trimmed[i+1][2][1]-trimmed[i+1][2][0]
            trimmed[i][1] = trimmed[i][2][1]-trimmed[i][2][0]
        i+=1
    
    trimmed[-1][-1][1] = sent_count #sent last number equal to end of podcast
    trimmed[-1][1] = trimmed[-1][-1][1] - trimmed[-1][-1][0]
    
    return condense(trimmed)

def full_condense(original, sent_len): #keep trimming with various lengths 
    condensed = copy.deepcopy(original)
    filled = trim_fill_gaps(condensed, 1, sent_len, condense_stream) 
    i=2
    #trim until max number of topics are reached, or all topics are alteast 2% of entire podcast
    while len(filled)>math.log(len(doc))*1.5 or any(count<len(sents)/50 for count in np.array(filled)[:,1]):
        filled = trim_fill_gaps(filled, i, sent_len, condense_stream)
        i+=1
    return filled

In [4]:
# confi = load_confidences(doc.text, dictionary, model, sents, condense_stream)
# for i in confi:
#     print(i)
dictionary, model = generate_model(doc.text, 2700)
condensed = condense_stream(load_confidences(doc.text, dictionary, model, sents)) #generate initial topics + confidences, then smooth by filling in empty values and averaging
print(len(condensed))
fill = full_condense(condensed, len(sents))
for ind, i in enumerate(fill):
    print(ind, i)
# for ind, i in enumerate(condensed):
#     print(i)


384


In [5]:
for i in condense_stream([section for section in condensed if section[1]>1]):
# for i in condense_stream(condensed):
    print(i)
    

[7.0, 2, [1, 3]]
[9.0, 171, [4, 175]]
[0.0, 7, [177, 184]]
[9.0, 2, [184, 186]]
[0.0, 164, [186, 350]]
[3.0, 108, [352, 460]]
[0.0, 9, [462, 471]]
[3.0, 88, [471, 559]]
[6.0, 2, [561, 563]]
[7.0, 101, [564, 665]]
[3.0, 6, [665, 671]]
[7.0, 7, [676, 683]]
[1.0, 5, [683, 688]]
[7.0, 24, [690, 714]]
[8.0, 3, [714, 717]]
[7.0, 26, [719, 745]]
[8.0, 47, [750, 797]]
[0.0, 2, [797, 799]]
[8.0, 135, [799, 934]]
[1.0, 5, [934, 939]]
[8.0, 21, [939, 960]]
[1.0, 2, [962, 964]]
[8.0, 5, [966, 971]]
[1.0, 203, [971, 1174]]
[0.0, 2, [1181, 1183]]
[1.0, 10, [1184, 1194]]
[7.0, 120, [1194, 1314]]
[1.0, 3, [1321, 1324]]
[7.0, 131, [1324, 1455]]
[6.0, 2, [1455, 1457]]
[7.0, 74, [1457, 1531]]
[8.0, 18, [1534, 1552]]
[7.0, 7, [1552, 1559]]
[8.0, 57, [1559, 1616]]
[7.0, 2, [1616, 1618]]
[8.0, 18, [1618, 1636]]
[0.0, 4, [1639, 1643]]
[8.0, 66, [1643, 1709]]
[6.0, 129, [1709, 1838]]
[9.0, 3, [1838, 1841]]
[6.0, 5, [1841, 1846]]


In [6]:
import copy
def trim_fill_gaps(sent_data, trim, sent_count, condense=lambda a: a):
    
    
    print("method prefilder",len(sent_data))
    trimmed = [section for section in sent_data if section[1]>trim] #keep sections of atleast a certain number of repeats
    print("method postfilter",len(sent_data))
#     for i in trimmed:
#         print(i)
#     for ind, i in enumerate(sent_data):
#         print(ind, i)


    trimmed[0][-1][0] = 0
    trimmed[0][1] = trimmed[0][-1][1] - trimmed[0][-1][0]
    i = 0
    while i <(len(trimmed)-1): #fill gaps between filtered numbers
        if trimmed[i][2][1]!=trimmed[i+1][2][0]:
            diff = (trimmed[i+1][2][0]-trimmed[i][2][1])/2
            print(trimmed[i], "\n",trimmed[i+1], diff, "sdfdsf\n")
    

            trimmed[i+1][2][0] -= int(diff)
            trimmed[i][2][1] += math.ceil(diff)

            trimmed[i+1][1] = trimmed[i+1][2][1]-trimmed[i+1][2][0]
            trimmed[i][1] = trimmed[i][2][1]-trimmed[i][2][0]
            print(trimmed[i], "\n",trimmed[i+1], "\n\n\n")
        i+=1
    
    trimmed[-1][-1][1] = sent_count
    trimmed[-1][1] = trimmed[-1][-1][1] - trimmed[-1][-1][0]

    
    
#     if trim!=0:
#         for i in sent_data:
#             print(i)
#         print("\t\t\tdone")
#     for ind, i in enumerate(condense(sent_data)):
#         print(ind, i)
#         print(a)
#     print("while done")
    
    return condense(trimmed)

def full_condense(original, sent_len):
    condensed = copy.deepcopy(original)
    print("start",len(condensed))
    filled = fill_gaps(condensed, 1, sent_len, condense_stream)
    
#     print("\t\t\t",1, len(condensed), len(filled))
#     filled = fill_gaps(filled, 2, sent_len, condense_stream)
#     print("\t\t\t",2, len(condensed), len(filled))
#     filled = fill_gaps(filled, 3, sent_len, condense_stream)
#     print("\t\t\t",3, len(condensed), len(filled))
#     for ind, j in enumerate(condense_stream(filled)):
#         print(j)
    i=2
    print("\t\t\t\tstart")
    
    while len(filled)>math.log(len(doc))*1.5 or any(count<len(sents)/50 for count in np.array(filled)[:,1]):
        print("\t\t\tbefore",i, len(filled))
        filled = fill_gaps(filled, i, sent_len, condense_stream)
        print("\t\t\tafter",i, len(filled))
#         for ind, j in enumerate(filled):
#             print(j)
        i+=1
    return filled

fill = full_condense(condensed, len(sents))
for ind, i in enumerate(condense_stream(condensed)):
    print(ind, i)

0 [9.0, 176, [0, 176]]
1 [0.0, 175, [176, 351]]
2 [3.0, 211, [351, 562]]
3 [7.0, 186, [562, 748]]
4 [8.0, 223, [748, 971]]
5 [1.0, 223, [971, 1194]]
6 [7.0, 339, [1194, 1533]]
7 [8.0, 176, [1533, 1709]]
8 [6.0, 137, [1709, 1846]]


In [ ]:
%%time

print("starting")
folder="3Lex"
podcast_host = "Lex Fridman"


# docs, doc_bin = process_folder_to_docs("5Lex", podcast_host)
onlyfiles=[(text_fix(open(folder+"/"+file).read()), file) for file in listdir(folder) if isfile(join(folder, file))]
docs = [] #list of docs 

for doc, name in tqdm(nlp.pipe(onlyfiles, as_tuples=True)): #piping all collection of docs to make doclist and docbin
    #each doc contains hostname, guest, title, entities mentioned, and summary
    doc.user_data["url"] = main+url+"/"+strip_accents(name.replace("|","-").replace("#","").replace(".txt","").replace("_","-").replace(",","").lower())
    name = re.split("[\|]",name)
    name=name[1:] if name[0][1:].isdigit() else name # store name of guest and topic, add to doc user data
    
    doc.user_data["host"] = podcast_host
    doc.user_data["guest"]= str(name[0]).replace("_"," ")
    doc.user_data["title"]= str(name[1][:-4]).replace("_"," ")
    docs.append(doc)

print("done")



In [ ]:
for ind, doc in enumerate(docs):
    print(ind, doc.user_data["guest"], doc.user_data["title"])

In [ ]:
docs[0].user_data["traits"].keys()

In [ ]:
import shutil
count = 0
not_processed = []
for ind, file in enumerate(os.listdir("7Lex")):
    if file.endswith(".txt"):
        file2 = file.replace("_", " ").replace("|", " ").replace(".txt", " ")
    #     print(ind, file) 
        processed = False


        for doc in docs:
            if doc.user_data["guest"] in file2 and doc.user_data["title"] in file2:
    #             os.rename("5Lex/"+file, "7Lex/"+file)
                
                processed = True
                count+=1
                break
        if processed:
            os.rename("7Lex/"+file, "5Lex/"+file)
print("done")

In [ ]:
import pymongo
from pymongo import MongoClient

username = urllib.parse.quote_plus('Empyr3an')
password = urllib.parse.quote_plus("B4ldr1c7@1")

cluster = pymongo.MongoClient("mongodb+srv://{}:{}@cluster0.4pec2.mongodb.net/Checkra?retryWrites=true&w=majority".format(username, password))
db = cluster["production"]
collection = db["lex"]

# db.create_collection("lmini")

In [ ]:
queried = list(db["lex"].find({},{"_id":0,"traits.Events":1}))
all_ents = [ent for doc in queried for ent in doc["traits"]["Events"]]

In [ ]:
db["lex"].drop()
db["lex"].insert_many([doc.user_data for doc in docs])

In [ ]:
all_atts = []
for doc in list(collection.find({"books":{"$ne":[]}})):
    print(doc["guest"], doc[att])
    for a in doc[att]:
        all_atts.append(a)

In [ ]:
%%time
#loading in previous bytestreams, needs 2 since 2 podcast folders edit later
doc_bin = DocBin(store_user_data=True) #docbin container for serialization

    doc_bin.add(doc) #add doc to list and bin
    
# with open("cucumber/Lexnumbered.bin", "wb") as f: #write bytestream to first_doc.bin
#     f.write(doc_bin.to_bytes())

with open("cucumber/Lex.bin", "rb") as w: 
    new_docs1 = DocBin(store_user_data=True).from_bytes(w.read())
docs1 = list(new_docs1.get_docs(nlp.vocab))

In [ ]:

import networkx as nx
import pandas as pd
att_to_csv(docs1, "people") #to get csv of edges

#generate megagraph for book ents
edges = pd.read_csv("booksEdges.csv", sep=',').drop("id", axis=1).values.tolist()
nodes = pd.read_csv('booksNodes.csv',sep=',').set_index("id").to_dict("index")

source = [nodes.get(e[0])["name"] for e in edges]
target = [nodes.get(e[1])["name"] for e in edges]
kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':np.ones(len(source))})
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())
plt.figure(figsize=(12,12))

pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

#following are other example graphviews

In [ ]:
G=nx.from_pandas_edgelist(kg_df[kg_df['target']=="Bible"], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()